In [3]:
import torch

from script.NeuralNets.Networks import SequentialNN, ICNN, ICNNApproxMax, ICNNLogical
from script.settings import device, data_type
import script.DHOV.MultiDHOV as multidhov
from script.Verification.Verifier import SingleNeuronVerifier, MILPVerifier, DHOVVerifier
import gurobipy as grp
from torchvision.datasets import CIFAR10
from torchvision.transforms import Compose, ToTensor, Normalize

In [23]:
def add_max_constr(model, neuron_name):
    neuron_var = model.getVarByName(neuron_name)
    model.setObjective(neuron_var, grp.GRB.MAXIMIZE)

def add_min_constr(model, neuron):
    neuron_var = model.getVarByName(neuron)
    model.setObjective(neuron_var, grp.GRB.MINIMIZE)

In [24]:
def optimize_model(model, neuron_name):
    model.update()
    model.optimize()
    if model.Status == grp.GRB.OPTIMAL:
        print("opt value: {}".format(model.getVarByName(neuron_name).getAttr("x")))

In [25]:
def icnn_model(icnn, nn, input_x, eps, layer_index, from_neuron, to_neuron, print_log=False):
    m = grp.Model()
    if not print_log:
        m.Params.LogToConsole = 0

    input_flattened = torch.flatten(input_x)
    bounds_affine_out, bounds_layer_out = nn.calculate_box_bounds(
        [input_flattened.add(-eps), input_flattened.add(eps)])

    parameter_list = list(nn.parameters())

    input_size = len(parameter_list[layer_index])
    lb = bounds_layer_out[layer_index][0].detach().cpu().numpy()
    ub = bounds_layer_out[layer_index][1].detach().cpu().numpy()
    in_var = m.addMVar(input_size, lb=lb, ub=ub, name="icnn_var")

    low = bounds_layer_out[layer_index][0][from_neuron: to_neuron]
    up = bounds_layer_out[layer_index][1][from_neuron: to_neuron]
    constraint_bounds_affine_out, constraint_bounds_layer_out = icnn.calculate_box_bounds([low, up])
    icnn.add_max_output_constraints(m, in_var[from_neuron: to_neuron], constraint_bounds_affine_out, constraint_bounds_layer_out)

    return m

In [41]:
"""W1 = [1. 1.; 1. -1.]
    b1 = [0., 0.]
    W2 = [1. 1.; 1. -1.]
    b2 = [-0.5, 0.]
    W3 = [-1. 1.; 1. 1.]
    b3 = [3., 0.] """

nn = SequentialNN([2, 2, 2, 2])

with torch.no_grad():
    parameter_list = list(nn.parameters())
    parameter_list[0].data = torch.tensor([[1, 1], [1, -1]], dtype=data_type).to(device)
    parameter_list[1].data = torch.tensor([0, 0], dtype=data_type).to(device)
    parameter_list[2].data = torch.tensor([[1, 1], [1, -1]], dtype=data_type).to(device)
    parameter_list[3].data = torch.tensor([-0.5, 0], dtype=data_type).to(device)
    parameter_list[4].data = torch.tensor([[-1, 1], [1, 1]], dtype=data_type).to(device)
    parameter_list[5].data = torch.tensor([3, 0], dtype=data_type).to(device)

test_image = torch.tensor([[0, 0]], dtype=data_type).to(device)

"""transform = Compose([ToTensor(),
                         Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
                        )

training_data = CIFAR10(root="../../cifar", train=True, download=True, transform=transform)
images, labels = training_data.__getitem__(0)
test_image, test_label = torch.unsqueeze(images, 0).to(dtype=data_type).to(device), torch.unsqueeze(
    torch.tensor(labels), 0).to(dtype=data_type).to(device)

nn = SequentialNN([32 * 32 * 3, 1024, 512, 10])
nn.load_state_dict(torch.load("../../cifar_fc.pth", map_location=torch.device(device)), strict=False)
parameter_list = list(nn.parameters())"""


'transform = Compose([ToTensor(),\n                         Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]\n                        )\n\ntraining_data = CIFAR10(root="../../cifar", train=True, download=True, transform=transform)\nimages, labels = training_data.__getitem__(0)\ntest_image, test_label = torch.unsqueeze(images, 0).to(dtype=data_type).to(device), torch.unsqueeze(\n    torch.tensor(labels), 0).to(dtype=data_type).to(device)\n\nnn = SequentialNN([32 * 32 * 3, 1024, 512, 10])\nnn.load_state_dict(torch.load("../../cifar_fc.pth", map_location=torch.device(device)), strict=False)\nparameter_list = list(nn.parameters())'

In [42]:
group_size = 2
eps = 1
icnns = []
for i in range((len(parameter_list) - 2) // 2):
    layer_index = int(i / 2)
    in_size = nn.layer_widths[layer_index + 1]
    icnns.append([])
    for k in range(in_size // group_size):
        next_net = ICNNLogical([group_size, 10, 10, 10, 1], force_positive_init=False, with_two_layers=False, init_scaling=10,
                                 init_all_with_zeros=False)
        icnns[i].append(next_net)
    if in_size % group_size > 0:
        next_net = ICNNLogical([in_size % group_size, 10, 10, 10, 1], force_positive_init=False, with_two_layers=False,
                               init_scaling=10,
                               init_all_with_zeros=False)
        icnns[i].append(next_net)

icnns = \
    multidhov.start_verification(nn, test_image, icnns, group_size, eps=eps, icnn_epochs=100, icnn_batch_size=1000,
                                 sample_count=1000, sample_new=False, use_over_approximation=True,
                                 sample_over_input_space=False, sample_over_output_space=True,
                                 force_inclusion_steps=0, preemptive_stop=False, even_gradient_training=False,
                                 keep_ambient_space=True, data_grad_descent_steps=0, train_outer=False,
                                 should_plot="none", optimizer="SdLBFGS", init_network=True, adapt_lambda="none")




approximation of layer: 0
layer progress, group 0 of 1 
        time for training: 3.29062819480896
        actual verification time 0.0708014965057373
        time for verification: 0.24431419372558594

approximation of layer: 1
layer progress, group 0 of 1 
        time for training: 4.470314979553223
        actual verification time 0.0705716609954834
        time for verification: 0.2339920997619629


In [43]:
input_flattened = torch.flatten(test_image)
bounds_affine_out, bounds_layer_out = nn.calculate_box_bounds([input_flattened.add(-eps), input_flattened.add(eps)])
print("bounds affine out \n {}".format(bounds_affine_out))
print("bounds layer out \n {}".format(bounds_layer_out))

bounds affine out 
 [[tensor([-2., -2.], dtype=torch.float64, grad_fn=<AddBackward0>), tensor([2., 2.], dtype=torch.float64, grad_fn=<AddBackward0>)], [tensor([-0.5000, -2.0000], dtype=torch.float64, grad_fn=<AddBackward0>), tensor([3.5000, 2.0000], dtype=torch.float64, grad_fn=<AddBackward0>)], [tensor([-0.5000,  0.0000], dtype=torch.float64, grad_fn=<AddBackward0>), tensor([5.0000, 5.5000], dtype=torch.float64, grad_fn=<AddBackward0>)]]
bounds layer out 
 [[tensor([0., 0.], dtype=torch.float64, grad_fn=<MaximumBackward0>), tensor([2., 2.], dtype=torch.float64, grad_fn=<MaximumBackward0>)], [tensor([0., 0.], dtype=torch.float64, grad_fn=<MaximumBackward0>), tensor([3.5000, 2.0000], dtype=torch.float64, grad_fn=<MaximumBackward0>)], [tensor([-0.5000,  0.0000], dtype=torch.float64, grad_fn=<AddBackward0>), tensor([5.0000, 5.5000], dtype=torch.float64, grad_fn=<AddBackward0>)]]


In [44]:
milp_verifier = MILPVerifier(nn, test_image, eps, print_log=False)
snv_verifier = SingleNeuronVerifier(nn, test_image, eps, print_log=False)
#dhov_verifier = DHOVVerifier(icnns, group_size, nn, test_image, 1)

milp_verifier.generate_constraints_for_net()
snv_verifier.generate_constraints_for_net()
#dhov_verifier.generate_constraints_for_net()

In [45]:
layer_index = 0
neuron_index = 0
milp_model = milp_verifier.model
snv_model = snv_verifier.model
#dhov_model = dhov_verifier.model
dhov_model = icnn_model(icnns[layer_index][0], nn, test_image, eps, layer_index, 0, 2, print_log=False)
milp_model.update()
snv_model.update()
dhov_model.update()
"""all_var = milp_model.getVars()
for var in all_var:
    print(var)"""
neuron_name = "relu_var{}[{}]".format(2*layer_index, neuron_index)
icnn_neuron_name = "icnn_var[{}]".format(neuron_index)

In [46]:
milp_copy = milp_model.copy()
snv_copy = snv_model.copy()
dhov_copy = dhov_model.copy()

"""milp_copy.Params.LogToConsole = 0
snv_copy.Params.LogToConsole = 0
dhov_copy.Params.LogToConsole = 0"""

add_max_constr(milp_copy, neuron_name)
add_max_constr(snv_copy, neuron_name)
add_max_constr(dhov_copy, icnn_neuron_name)

optimize_model(milp_copy, neuron_name)
print("===================================================================================")
optimize_model(snv_copy, neuron_name)
print("===================================================================================")
optimize_model(dhov_copy, icnn_neuron_name)
print("===================================================================================")

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 26 rows, 16 columns and 54 nonzeros
Model fingerprint: 0x62243f80
Variable types: 12 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [5e-01, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e-01, 6e+00]
  RHS range        [5e-01, 3e+00]
Presolve removed 14 rows and 6 columns
Presolve time: 0.00s
Presolved: 12 rows, 10 columns, 36 nonzeros
Variable types: 6 continuous, 4 integer (4 binary)
Found heuristic solution: objective -0.0000000

Root relaxation: objective 2.000000e+00, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       2.00

In [35]:
milp_copy = milp_model.copy()
snv_copy = snv_model.copy()
dhov_copy = dhov_model.copy()

milp_copy.Params.LogToConsole = 0
snv_copy.Params.LogToConsole = 0
dhov_copy.Params.LogToConsole = 0

add_min_constr(milp_copy, neuron_name) #affine_var0[0]
add_min_constr(snv_copy, neuron_name)
add_min_constr(dhov_copy, icnn_neuron_name)

optimize_model(milp_copy, neuron_name)
print("===================================================================================")
optimize_model(snv_copy, neuron_name)
print("===================================================================================")
optimize_model(dhov_copy, icnn_neuron_name)

max value: 0.0
max value: 0.0
max value: 0.0
